# Full Data Preparation Pipeline
최종 산출물 : `review_business_5up_5aspect_3sentiment_vectorized_clean.json`

In [3]:
import pandas as pd
import json
import re
import os
from collections import defaultdict
from tqdm import tqdm
from transformers import DebertaV2Tokenizer

In [5]:
# --- Step 1: business.json preprocessing ---
df_B = pd.read_json("data/raw/yelp_academic_dataset_business.json", lines=True)
business_df = df_B.copy()
drop_cols = ['postal_code','latitude','longitude','attributes','hours']
business_df = business_df.drop(columns=drop_cols)
business_df.loc[business_df['city'].str.lower().str.contains("philadelphia", na=False),'city'] = "Philadelphia"

def load_categories(fp):
    with open(fp,'r',encoding='utf-8') as f:
        return set(line.strip().lower() for line in f if line.strip())
food_categories = load_categories('data/raw/food.txt')
restaurant_categories = load_categories('data/raw/restaurant.txt')
target_categories = food_categories.union(restaurant_categories)

def category_match(row):
    if isinstance(row,str):
        biz_categories = set(cat.strip().lower() for cat in row.split(','))
        return bool(biz_categories & target_categories)
    return False
business_food_df = business_df[business_df['categories'].apply(category_match)]
top_state = business_food_df['state'].value_counts().idxmax()
business_pa_df = business_food_df[business_food_df['state']==top_state]
business_paph_df = business_pa_df[business_pa_df['city']=="Philadelphia"]
mask = business_paph_df.apply(lambda col: col.map(lambda x: pd.isna(x) or (isinstance(x,str) and x.strip()==""))).any(axis=1)
business_paph_df_2 = business_paph_df[~mask].reset_index(drop=True)
business_paph_df_2.to_json("data/output/business.json", orient="records", lines=True, force_ascii=False)
print(f"총 {len(business_paph_df_2)}개 항목이 'data/output/business.json'에 저장되었습니다.")

총 6855개 항목이 'data/output/business.json'에 저장되었습니다.


In [8]:
# --- Step 1: review.json preprocessing ---
chunk_size=100000
chunks=pd.read_json("data/raw/yelp_academic_dataset_review.json", lines=True, chunksize=chunk_size)
df_review=pd.concat(chunk for chunk in chunks)
business_ids=set(business_paph_df_2['business_id'])
df_review=df_review[df_review['business_id'].isin(business_ids)]
df_review=df_review.drop(columns=['funny','cool'])

tqdm.pandas()
tokenizer=DebertaV2Tokenizer.from_pretrained("microsoft/deberta-v3-base")
df_review['token_length']=df_review['text'].progress_apply(lambda x: len(tokenizer.tokenize(x)))
df_review.to_json("data/output/review.json", orient="records", lines=True, force_ascii=False)

100%|██████████| 730552/730552 [02:22<00:00, 5126.15it/s]


In [9]:
# --- Step 1: user.json preprocessing ---
chunks=pd.read_json("data/raw/yelp_academic_dataset_user.json", lines=True, chunksize=100000)
df_user=pd.concat(chunk for chunk in chunks)
drop_columns=['yelping_since','funny','cool','elite','friends','fans','compliment_hot','compliment_more','compliment_profile','compliment_cute','compliment_list','compliment_note','compliment_plain','compliment_cool','compliment_funny','compliment_writer','compliment_photos']
df_user=df_user.drop(columns=drop_columns)
review_counts=df_review['user_id'].value_counts()
user_ids_5plus=review_counts[review_counts>=5].index
df_user=df_user[df_user['user_id'].isin(user_ids_5plus)]
df_user.to_json("data/output/user.json", orient="records", lines=True, force_ascii=False)

In [10]:
# --- Step 2: merge review, user and business ---
def load_jsonl(path):
    with open(path, 'r', encoding='utf-8') as f:
        return [json.loads(line) for line in f]

reviews = load_jsonl("data/output/review.json")
users = load_jsonl("data/output/user.json")
businesses = load_jsonl("data/output/business.json")

user_dict = {u['user_id']: u for u in users}
business_dict = {b['business_id']: b for b in businesses}

merged_data = []
for r in tqdm(reviews, desc='병합 중'):
    uid = r['user_id']
    bid = r['business_id']
    if uid in user_dict and bid in business_dict:
        m = r.copy()
        for k, v in user_dict[uid].items():
            m[f'user_{k}'] = v
        for k, v in business_dict[bid].items():
            m[f'business_{k}'] = v
        merged_data.append(m)

# DataFrame으로 변환
df = pd.DataFrame(merged_data)
# 불필요한 ID 컬럼 제거
df.drop(columns=["user_user_id", "business_business_id"], inplace=True, errors="ignore")
# 컬럼 이름 리네이밍
df.rename(columns={
    "stars": "review_stars",
    "useful": "review_useful",
    "date": "review_date"}, inplace=True)
# 저장 경로
output_path = "data/output/merged_dataset.json"
os.makedirs(os.path.dirname(output_path), exist_ok=True)
# JSONL 형식으로 저장
with open(output_path, "w", encoding="utf-8") as f:
    for row in df.to_dict(orient="records"):
        json.dump(row, f, ensure_ascii=False)
        f.write("\n")

print("병합 완료: merged_dataset.json")

병합 중: 100%|██████████| 730552/730552 [00:02<00:00, 317848.01it/s]


병합 완료: merged_dataset.json


In [ ]:
# --- Step 2.5: ABSA 감성 분석 수행 (DeBERTa v3 기반) ---
import json, os, torch, psutil, GPUtil
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from tqdm import tqdm

# 디바이스 설정
device = torch.device(
    "cuda" if torch.cuda.is_available() else
    "mps" if torch.backends.mps.is_available() else
    "cpu"
)
print(f"사용 디바이스: {device}")

# 메모리 상태 출력 함수
def print_memory_status():
    try:
        if torch.cuda.is_available():
            gpu = GPUtil.getGPUs()[0]
            print(f"\n[GPU 메모리] {gpu.memoryUsed}MB / {gpu.memoryTotal}MB | 사용률: {gpu.memoryUtil*100:.1f}% | 온도: {gpu.temperature}°C")
            print(f"[PyTorch 메모리] 할당: {torch.cuda.memory_allocated()/1024**2:.1f}MB / 캐시: {torch.cuda.memory_reserved()/1024**2:.1f}MB")
        memory = psutil.virtual_memory()
        print(f"[시스템 메모리] {memory.used/1024**2:.1f}MB / {memory.total/1024**2:.1f}MB | 사용률: {memory.percent}%")
    except Exception as e:
        print(f"메모리 확인 오류: {e}")

# 모델 및 토크나이저 로드
MODEL_NAME = "yangheng/deberta-v3-base-absa-v1.1"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME).to(device).eval()

# 분석할 측면
ASPECTS = ["food", "service", "price", "ambience", "location"]

# 감성 분석 함수
def analyze_sentiment(text, aspect):
    try:
        inputs = tokenizer(text, aspect, return_tensors="pt", truncation=True, max_length=512)
        inputs = {k: v.to(device) for k, v in inputs.items()}
        with torch.no_grad():
            logits = model(**inputs).logits
            probs = torch.softmax(logits, dim=1)[0]
        return {"scores": {model.config.id2label[i]: float(probs[i]) for i in range(len(probs))}, "status": "success"}
    except Exception as e:
        print(f"오류 발생: {e} (텍스트: {text[:50]}...)")
        return {"scores": None, "status": "error", "error_message": str(e)}

# 파일 경로 설정
input_file = "data/output/merged_dataset.json"
output_file = "data/output/review_5up_5aspect_3sentiment.jsonl"

# 기존 처리 리뷰 ID 불러오기
processed_ids = set()
if os.path.exists(output_file):
    with open(output_file, "r", encoding="utf-8") as f:
        for line in f:
            try:
                obj = json.loads(line)
                processed_ids.add(obj["review_id"])
            except:
                continue
    print(f"{len(processed_ids)}개 리뷰는 건너뜁니다.")

# 리뷰 처리 및 저장
buffer, count, skipped = [], 0, 0
save_every = 500

with open(input_file, "r", encoding="utf-8") as fin:
    for line in tqdm(fin, desc="ABSA 감성 분석 중"):
        try:
            obj = json.loads(line.strip())
            rid, text = obj.get("review_id"), obj.get("text", "")
            if not text or rid in processed_ids:
                skipped += 1
                continue
            sentiment = {asp: analyze_sentiment(text, asp) for asp in ASPECTS}
            buffer.append({"review_id": rid, "text": text, "sentiment": sentiment})
            count += 1
            if len(buffer) >= save_every:
                with open(output_file, "a", encoding="utf-8") as fout:
                    for r in buffer:
                        fout.write(json.dumps(r, ensure_ascii=False) + "\n")
                print(f"{count}개 리뷰 저장됨")
                print_memory_status()
                buffer = []
        except Exception as e:
            print(f"리뷰 처리 중 오류 발생: {e}")
            continue

# 남은 리뷰 저장
if buffer:
    with open(output_file, "a", encoding="utf-8") as fout:
        for r in buffer:
            fout.write(json.dumps(r, ensure_ascii=False) + "\n")

print(f"총 {count}개 리뷰 분석 완료, {skipped}개 건너뜀 → {output_file}")
print_memory_status()

In [17]:
# --- 입력 파일 경로 ---
merged_path = "data/output/merged_dataset.json"
absa_path = "data/output/review_5up_5aspect_3sentiment.jsonl"
final_output_path = "data/output/review_business_5up_5aspect_3sentiment_vectorized_clean.json"

# --- Step 1: 5개 이상 리뷰 작성 유저 필터링 ---
user_review_counts = defaultdict(int)
merged_data = []
with open(merged_path, "r", encoding="utf-8") as f:
    for line in f:
        obj = json.loads(line)
        user_review_counts[obj["user_id"]] += 1
        merged_data.append(obj)

qualified_users = {u for u, c in user_review_counts.items() if c >= 5}
merged_filtered = [obj for obj in merged_data if obj["user_id"] in qualified_users]
print(f"Step 1 완료: 총 {len(merged_data):,}건 중 리뷰 5개 이상 유저 리뷰 {len(merged_filtered):,}건 유지")

# --- Step 2: review_id → user 정보 맵 생성 ---
id_map = {}
user_biz_ids = defaultdict(set)

for obj in merged_filtered:
    rid = obj["review_id"]
    uid = obj["user_id"]
    bid = obj["business_id"]
    id_map[rid] = {
        "user_id": uid,
        "business_id": bid,
        "stars": obj["review_stars"],
        "review_useful": obj["review_useful"],
        "review_date": obj["review_date"],
    }
    user_biz_ids[uid].add(bid)

# --- Step 3: ABSA 결과 결합 + 벡터화 + 최종 필터링 ---
def sentiment_to_vector(sentiment_dict):
    aspects = ["food", "service", "price", "ambience", "location"]
    polarities = ["Negative", "Neutral", "Positive"]
    vector = []
    for asp in aspects:
        scores = sentiment_dict.get(asp, {}).get("scores", {})
        for pol in polarities:
            vector.append(scores.get(pol, 0.0))
    return vector

final_data = []
not_in_merged = 0
too_few_businesses = 0
decode_errors = 0

with open(absa_path, "r", encoding="utf-8") as f:
    for line in tqdm(f, desc="최종 벡터화 및 필터링"):
        try:
            obj = json.loads(line)
            rid = obj.get("review_id")
            if rid not in id_map:
                not_in_merged += 1
                continue
            info = id_map[rid]
            uid = info["user_id"]
            if len(user_biz_ids[uid]) < 5:
                too_few_businesses += 1
                continue
            vec = sentiment_to_vector(obj.get("sentiment", {}))
            final_data.append({
                "review_id": rid,
                "user_id": uid,
                "business_id": info["business_id"],
                "stars": info["stars"],
                "review_date": info["review_date"],
                "sentiment_vector": vec,
            })
        except json.JSONDecodeError:
            decode_errors += 1
            continue

# --- 최종 저장 ---
with open(final_output_path, "w", encoding="utf-8") as fout:
    for obj in final_data:
        fout.write(json.dumps(obj, ensure_ascii=False) + "\n")

# --- 통계 출력 ---
print(f"\n총 결과 리뷰 수: {len(final_data):,}건 저장 완료 → {final_output_path}")
print(f"- 병합되지 않은 리뷰: {not_in_merged:,}건")
print(f"- business 방문 수 5개 미만으로 제외된 리뷰: {too_few_businesses:,}건")
print(f"- JSON 디코딩 실패: {decode_errors:,}건")

Step 1 완료: 총 451,185건 중 리뷰 5개 이상 유저 리뷰 451,185건 유지


최종 벡터화 및 필터링: 452505it [00:09, 45375.70it/s]



총 결과 리뷰 수: 447,796건 저장 완료 → data/output/review_business_5up_5aspect_3sentiment_vectorized_clean.json
- 병합되지 않은 리뷰: 1,320건
- business 방문 수 5개 미만으로 제외된 리뷰: 3,389건
- JSON 디코딩 실패: 0건


In [ ]:
input_path = "data/output/review_business_5up_5aspect_3sentiment_vectorized_clean.json"
absa_path = "data/output/review_5up_5aspect_3sentiment.jsonl"
output_path = "data/output/review_business_5up_with_text.json"

review_text_map = {}
with open(absa_path, "r", encoding="utf-8") as f:
    for line in f:
        try:
            obj = json.loads(line)
            review_text_map[obj["review_id"]] = obj.get("text", "")
        except:
            continue

# 입력 파일 읽어서 text 추가, sentiment_vector 제거
with open(input_path, "r", encoding="utf-8") as fin, open(output_path, "w", encoding="utf-8") as fout:
    for line in tqdm(fin, desc="텍스트 추가 중"):
        try:
            obj = json.loads(line)
            rid = obj["review_id"]
            obj["text"] = review_text_map.get(rid, "")
            obj.pop("sentiment_vector", None)
            fout.write(json.dumps(obj, ensure_ascii=False) + "\n")
        except Exception as e:
            print(f"처리 오류: {e}")
            continue

print(f"완료: 텍스트 포함 리뷰가 '{output_path}'에 저장")
